# scikit-learn 支持向量机算法库使用小结

之前通过一个系列对支持向量机(以下简称SVM)算法的原理做了一个总结，本文从实践的角度对scikit-learn SVM算法库的使用做一个小结。scikit-learn SVM算法库封装了libsvm 和 liblinear 的实现，仅仅重写了算法了接口部分。

## 1. scikit-learn SVM算法库使用概述

scikit-learn中SVM的算法库分为两类，一类是分类的算法库，包括SVC， NuSVC，和LinearSVC 3个类。另一类是回归算法库，包括SVR， NuSVR，和LinearSVR 3个类。相关的类都包裹在sklearn.svm模块之中。

对于SVC， NuSVC，和LinearSVC 3个分类的类，SVC和 NuSVC差不多，区别仅仅在于对损失的度量方式不同，而LinearSVC从名字就可以看出，他是线性分类，也就是不支持各种低维到高维的核函数，仅仅支持线性核函数，对线性不可分的数据不能使用。

同样的，对于SVR， NuSVR，和LinearSVR 3个回归的类， SVR和NuSVR差不多，区别也仅仅在于对损失的度量方式不同。LinearSVR是线性回归，只能使用线性核函数。

我们使用这些类的时候，如果有经验知道数据是线性可以拟合的，那么使用LinearSVC去分类 或者LinearSVR去回归，它们不需要我们去慢慢的调参去选择各种核函数以及对应参数， 速度也快。如果我们对数据分布没有什么经验，一般使用SVC去分类或者SVR去回归，这就需要我们选择核函数以及对核函数调参了。

什么特殊场景需要使用NuSVC分类 和 NuSVR 回归呢？如果我们对训练集训练的错误率或者说支持向量的百分比有要求的时候，可以选择NuSVC分类 和 NuSVR 。它们有一个参数来控制这个百分比。

这些类的详细使用方法我们在下面再详细讲述。

## 2. 回顾SVM分类算法和回归算法

我们先简要回顾下SVM分类算法和回归算法，因为这里面有些参数对应于算法库的参数，如果不先复习下，下面对参数的讲述可能会有些难以理解。

对于SVM分类算法，其原始形式是：

$$min\frac{1}{2}||w||_2^2+C\displaystyle\sum_{i=1}^{m}ξ_i$$

$$s.t. \quad y_i(w∙ϕ(x_i)+b)≥1−ξ_i(i=1,2,...m)$$

$$ξ_i≥0(i=1,2,...m)$$

其中m为样本个数，我们的样本为$$(x_1,y_1),(x_2,y_2),...,(x_m,y_m)$$。

$$w,b$$是我们的分离超平面的$$w∙ϕ(x_i)+b=0$$系数, $$ξ_i$$为第$$i$$个样本的松弛系数，C为惩罚系数。

$$ϕ(x_i)$$为低维到高维的映射函数。

通过拉格朗日函数以及对偶化后的形式为：

$$\underbrace {min}_{α}\frac{1}{2} \displaystyle\sum_{i=1,j=1}^{m}α_iα_jy_iy_jK(x_i,x_j)−\displaystyle\sum_{i=1}^{m}α_i$$

$$s.t. \quad \displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$0≤α_i≤C$$

其中和原始形式不同的α为拉格朗日系数向量。$$K(x_i,x_j)$$为我们要使用的核函数。

对于SVM回归算法，其原始形式是：

$$min\frac{1}{2}||w||_2^2+C\displaystyle\sum_{i=1}^{m}(ξ_i^∨+ξ_i^∧)$$

$$s.t. \quad −ϵ−ξ∨_i≤y_i−w∙ϕ(x_i)−b≤ϵ+ξ_i^∧$$

$$ξ_i^∨≥0,ξ_i^∧≥0(i=1,2,...,m)$$

其中m为样本个数，我们的样本为$$(x_1,y_1),(x_2,y_2),...,(x_m,y_m)$$。w,b是我们的回归超平面的$$w∙x_i+b=0$$系数, $$ξ_i^∨，ξ_i^∧$$为第$$i$$个样本的松弛系数， $$C$$为惩罚系数，$$ϵ$$为损失边界，到超平面距离小于$$ϵ$$的训练集的点没有损失。$$ϕ(x_i)$$为低维到高维的映射函数。

通过拉格朗日函数以及对偶化后的形式为：

$$\underbrace {min}_{α^∨,α^∧}\frac{1}{2}\displaystyle\sum_{i=1,j=1}^{m}(α_i^∧−α_i^∨)(α_j^∧−α_j^∨)K_{ij}+\displaystyle\sum_{i=1}^{m}(ϵ−y_i)α_i^∧+(ϵ+y_i)α_i^∨$$

$$s.t. \quad \displaystyle\sum_{i=1}^{m}(α_i^∧−α_i^∨)=0$$

$$0<α_i^∨<C(i=1,2,...m)$$

$$0<α_i^∧<C(i=1,2,...m)$$

其中和原始形式不同的$$α^∨，α^∧$$为拉格朗日系数向量。$$K(x_i,x_j)$$为我们要使用的核函数。

## 3. SVM核函数概述

在scikit-learn中，内置的核函数一共有4种，当然如果你认为线性核函数不算核函数的话，那就只有三种。

1）线性核函数（Linear Kernel）表达式为：$$K(x,z)=x∙z$$，就是普通的内积，LinearSVC 和 LinearSVR 只能使用它。

2)  多项式核函数（Polynomial Kernel）是线性不可分SVM常用的核函数之一，表达式为：$$K(x,z)=（γx∙z+r)^d$$ ，其中，$$γ,r,d$$都需要自己调参定义,比较麻烦。

3）高斯核函数（Gaussian Kernel），在SVM中也称为径向基核函数（Radial Basis Function,RBF），它是libsvm默认的核函数，当然也是scikit-learn默认的核函数。表达式为：$$K(x,z)=exp(−γ||x−z||^2)$$， 其中，$$γ$$大于0，需要自己调参定义。

4）Sigmoid核函数（Sigmoid Kernel）也是线性不可分SVM常用的核函数之一，表达式为：$$K(x,z)=tanh（γx∙z+r)$$， 其中，$$γ,r$$都需要自己调参定义。

一般情况下，对非线性数据使用默认的高斯核函数会有比较好的效果，如果你不是SVM调参高手的话，建议使用高斯核来做数据分析。　　

## 4. SVM分类算法库参数小结

这里我们对SVM分类算法库的重要参数做一个详细的解释，重点讲述调参的一些注意点。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537944305086_lZ5ko9Ce5L.jpg)

## 5. SVM回归算法库参数小结

SVM回归算法库的重要参数巨大部分和分类算法库类似，因此这里重点讲述和分类算法库不同的部分，对于相同的部分可以参考上一节对应参数。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537948319698_IpYePgtDRg.jpg)

## 6. SVM算法库其他调参要点

上面已经对scikit-learn中类库的参数做了总结，这里对其他的调参要点做一个小结。

1）一般推荐在做训练之前对数据进行归一化，当然测试集中的数据也需要归一化。。

2）在特征数非常多的情况下，或者样本数远小于特征数的时候，使用线性核，效果已经很好，并且只需要选择惩罚系数C即可。

3）在选择核函数时，如果线性拟合不好，一般推荐使用默认的高斯核'rbf'。这时我们主要需要对惩罚系数C和核函数参数γ进行艰苦的调参，通过多轮的交叉验证选择合适的惩罚系数C和核函数参数γ。

4）理论上高斯核不会比线性核差，但是这个理论却建立在要花费更多的时间来调参上。所以实际上能用线性核解决问题我们尽量使用线性核。

文章转载自:[博客园 刘建平Pinard scikit-learn 支持向量机算法库使用小结](https://www.cnblogs.com/pinard/p/6117515.html)

